In [5]:

!pip install -q langchain openai langchain-community langchain-openai

# Set API key
import os
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, tool
import requests
os.environ["OPENROUTER_API_KEY"] = "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"

# Set up LLM from OpenRouter
# Claude 3 Haiku model
# Sets the model to be deterministic so no randomness in responses.
# Points to OpenRouter’s API endpoint, which proxies requests to the Claude model.
llm = ChatOpenAI(
    model="anthropic/claude-3-haiku",
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENROUTER_API_KEY"]
)

# Write a API which can get the price for a stock based on the start and end date , number of days etc.
# https://api.financialdatasets.ai/prices/?ticker={ticker}.
# And fetch the financial metrics: https://api.financialdatasets.ai/financial-metrics

# Function to get stock prices
def get_stock_prices(ticker: str, interval="day", interval_multiplier=1, start_date=None, end_date=None):
    url = "https://api.financialdatasets.ai/prices/"
    params = {
        "ticker": ticker,
        "interval": interval,
        "interval_multiplier": interval_multiplier,
        "start_date": start_date,
        "end_date": end_date
    }
    # Remove any None values
    params = {k: v for k, v in params.items() if v is not None}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

# Function to get financial metrics for a stock
def get_financial_metrics(ticker: str, period="annual"):
    url = "https://api.financialdatasets.ai/financial-metrics"
    params = {"ticker": ticker, 
              "period": period
             }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None


# Example:
aapl_prices = get_stock_prices("AAPL", interval="day", interval_multiplier=1, start_date="2023-01-01", end_date="2023-01-10")
print("AAPL Prices:", aapl_prices)

aapl_metrics = get_financial_metrics("AAPL", period="ttm")
print("AAPL Financial Metrics:", aapl_metrics)


AAPL Prices: {'ticker': 'AAPL', 'prices': [{'ticker': 'AAPL', 'open': 130.28, 'close': 125.07, 'high': 130.9, 'low': 124.17, 'volume': 112117471, 'time': '2023-01-03T05:00:00Z', 'time_milliseconds': 1672722000000}, {'ticker': 'AAPL', 'open': 126.89, 'close': 126.36, 'high': 128.66, 'low': 125.08, 'volume': 89100633, 'time': '2023-01-04T05:00:00Z', 'time_milliseconds': 1672808400000}, {'ticker': 'AAPL', 'open': 127.13, 'close': 125.02, 'high': 127.77, 'low': 124.76, 'volume': 80716808, 'time': '2023-01-05T05:00:00Z', 'time_milliseconds': 1672894800000}, {'ticker': 'AAPL', 'open': 126.01, 'close': 129.62, 'high': 130.29, 'low': 124.89, 'volume': 87754715, 'time': '2023-01-06T05:00:00Z', 'time_milliseconds': 1672981200000}, {'ticker': 'AAPL', 'open': 130.47, 'close': 130.15, 'high': 133.41, 'low': 129.89, 'volume': 70790813, 'time': '2023-01-09T05:00:00Z', 'time_milliseconds': 1673240400000}, {'ticker': 'AAPL', 'open': 130.26, 'close': 130.73, 'high': 131.26, 'low': 128.12, 'volume': 6389

In [6]:
def get_line_items(ticker):
    url = "https://api.financialdatasets.ai/financials/search/line-items"
    params = {"ticker": ticker}
    response = requests.get(url, params=params)
    return response.json()

def get_insider_trades(ticker):
    url = "https://api.financialdatasets.ai/insider-trades"
    params = {"ticker": ticker}
    response = requests.get(url, params=params)
    return response.json()

def get_news(ticker):
    url = "https://api.financialdatasets.ai/news/"
    params = {"ticker": ticker}
    response = requests.get(url, params=params)
    return response.json()

def get_company_facts(ticker):
    url = "https://api.financialdatasets.ai/company/facts/"
    params = {"ticker": ticker}
    response = requests.get(url, params=params)
    return response.json()


In [11]:
import re, json
from pydantic import BaseModel
from tenacity import retry, stop_after_attempt, wait_fixed
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

class Price(BaseModel):
    open: float
    close: float
    high: float
    low: float
    volume: int
    time: str

@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def get_price_summary(ticker: str) -> Price:
    prompt = (
        f"Give me the open, close, high, low, volume, and time for "
        f"{ticker.upper()} in JSON"
    )
    response = llm.invoke([HumanMessage(content=prompt)]).content
    
    if not re.search(r"\{[\s\S]*?\}", response):
        raise ValueError("Error in LLM response")
    data = json.loads(re.search(r"\{[\s\S]*?\}", response).group())
    clean_data = Price(**data)
    return clean_data


# Example
print(get_price_summary("AAPL"))

open=159.58 close=160.55 high=160.77 low=159.01 volume=78400000 time='2023-04-26 16:00:00'
